In [1]:
import io
import pandas as pd

In [2]:
data = """costCenter: LL63238012
mail: shiva.gowni@LLp.com
LLpResponsible: cn=LLf58420,ou=Personal,ou=People,ou=LLDI,o=LLP
LLpHomeDirectory: nisMapName=auto.home,ou=RDC_AMEC_LL-CDC01,ou=Locations,ou=LLDI,o=LLP#0#Quality=economy,NisMap=llc1002:/proj/llc1002_ziz1/q,Quota=10621
LLpHomeDirectory: nisMapName=auto.home,ou=RDC_AMEC_LL-CDC01,ou=Locations,ou=LLDI,o=LLP#0#Quality=scratKG,NisMap=llc1002:/proj/llc1002_ziz1_scratKG/q,Quota=12000,Id=scratKG
fullName: Tulip project ziz1

costCenter: MX61FRK604
mail: ali.pina@LLp.com
LLpResponsible: cn=LLa11826,ou=Personal,ou=People,ou=LLDI,o=LLP
LLpHomeDirectory: nisMapName=auto.home,ou=SAT_AMEC_MX-GDL01,ou=Locations,ou=LLDI,o=LLP#0#Quality=reference,NisMap=llc0156:/proj/llc0156_zmx28home_3/q,Quota=100,Id=3
LLpHomeDirectory: nisMapName=auto.home,ou=SAT_AMEC_MX-GDL01,ou=Locations,ou=LLDI,o=LLP#0#Quality=reference,NisMap=llc0156:/proj/llc0156_zmx28home/q,Quota=300
LLpHomeDirectory: nisMapName=auto.home,ou=SAT_AMEC_MX-GDL01,ou=Locations,ou=LLDI,o=LLP#0#Quality=reference,NisMap=llc0156:/proj/llc0156_zmx28home_2/q,Quota=100,Id=2
fullName: xFSL to LLDI migration



costCenter: RU61FPD561
mail: udi.landen@LLp.com
LLpResponsible: cn=LLa09278,ou=Personal,ou=People,ou=LLDI,o=LLP
LLpHomeDirectory: nisMapName=auto.home,ou=RDC_AMEC_LL-CDC01,ou=Locations,ou=LLDI,o=LLP#0#Quota=1,Quality=EconomyHP,NisMap=llc2002:/proj/llc2002_zru12/q
LLpHomeDirectory: nisMapName=auto.home,ou=RDC_AMEC_LL-CDC01,ou=Locations,ou=LLDI,o=LLP#0#Quota=2800,Quality=EconomyHP,NisMap=llc1002:/proj/llc1002_zru12_analog/q,Id=analog
LLpHomeDirectory: nisMapName=auto.home,ou=RDC_AMEC_LL-CDC01,ou=Locations,ou=LLDI,o=LLP#0#Quota=1100,Quality=EconomyHP,NisMap=llc1002:/proj/llc1002_zru12_home/q,Id=home
LLpHomeDirectory: nisMapName=auto.home,ou=RDC_AMEC_LL-CDC01,ou=Locations,ou=LLDI,o=LLP#0#Quality=EconomyHP,NisMap=llc1002:/proj/llc1002_zru12_libddk/q,Quota=2162,Id=libddk
LLpHomeDirectory: nisMapName=auto.home,ou=RDC_AMEC_LL-CDC01,ou=Locations,ou=LLDI,o=LLP#0#Quality=EconomyHP,NisMap=llc1002:/proj/llc1002_zru12_proj/q,Quota=1102,Id=proj
fullName: zru12

costCenter: KG63010285
mail: adam.smith@LLp.com
LLpHomeDirectory: nisMapName=auto.home,ou=RDC_EMEA_NL-CDC01,ou=Locations,ou=LLDI,o=LLP#0#Quality=BLLinessCriticalHP,Quota=60,NisMap=llc4008:/proj/llc4008_zuriKG/q
fullName: Container to store ZuriKG vault



costCenter: KG63010285
mail: adam.smith@LLp.com
LLpHomeDirectory: nisMapName=auto.home,ou=RDC_EMEA_NL-CDC01,ou=Locations,ou=LLDI,o=LLP#0#Quality=EconomyHP,Quota=1,NisMap=llc3008:/proj/llc3008_zuriKG_rme/q
LLpHomeDirectory: nisMapName=auto.home,ou=SAT_EMEA_NL-RME01,ou=Locations,ou=LLDI,o=LLP#0#Quality=EconomyHP,Quota=30,NisMap=llc4014:/proj/llc4014_zuriKG_rme/q
LLpHomeDirectory: nisMapName=auto.home,ou=SAT_EMEA_NL-RME01,ou=Locations,ou=LLDI,o=LLP#0#Quality=ScratKGHP,Quota=400,NisMap=llc4014:/proj/llc4014_zuriKG_rme_scratKG/q,Id=scratKG
fullName: Project to restore  project data on the RME work on HP-UX


"""

https://stackoverflow.com/questions/73693154/pandas-to-extract-directory-data-into-tabular-form

In [3]:
def generate(data):
    for record in data.split("\n\n"):              # Split records based on two newlines (unix)
        result = {}
        for line in record.split("\n"):            # Split properties based on single newlines (unix)
            if line:                               # Skip empty lines happening for extra or trailing newlines
                key, *value = line.split(": ")     # Tolerant to lines with more than a single ´: ´ (*values)
                value = ": ".join(value)           # Recover original value if more than a single (`: `)
                if key in result:
                    result[key] += ";" + value
                else:
                    result[key] = value
        if result:                                  # Don't yield empty results
            yield result

In [4]:
frame = pd.DataFrame(generate(data))

In [5]:

frame["responsible"] = frame["LLpResponsible"].str.extract("cn=([\w]*)")
frame["location"] = frame["LLpHomeDirectory"].str.extract("ou=([\w_\-]*)")
frame["directory"] = frame["LLpHomeDirectory"].str.findall("NisMap=\w+:([\w_\-/]*)")

In [6]:
frame.loc[0,["LLpHomeDirectory"]].values

array(['nisMapName=auto.home,ou=RDC_AMEC_LL-CDC01,ou=Locations,ou=LLDI,o=LLP#0#Quality=economy,NisMap=llc1002:/proj/llc1002_ziz1/q,Quota=10621;nisMapName=auto.home,ou=RDC_AMEC_LL-CDC01,ou=Locations,ou=LLDI,o=LLP#0#Quality=scratKG,NisMap=llc1002:/proj/llc1002_ziz1_scratKG/q,Quota=12000,Id=scratKG'],
      dtype=object)

In [7]:
frame

,costCenter,mail,LLpResponsible,LLpHomeDirectory,fullName,responsible,location,directory
0,LL63238012,shiva.gowni@LLp.com,"cn=LLf58420,ou=Personal,ou=People,ou=LLDI,o=LLP","nisMapName=auto.home,ou=RDC_AMEC_LL-CDC01,ou=L...",Tulip project ziz1,LLf58420,RDC_AMEC_LL-CDC01,"[/proj/llc1002_ziz1/q, /proj/llc1002_ziz1_scra..."
1,MX61FRK604,ali.pina@LLp.com,"cn=LLa11826,ou=Personal,ou=People,ou=LLDI,o=LLP","nisMapName=auto.home,ou=SAT_AMEC_MX-GDL01,ou=L...",xFSL to LLDI migration,LLa11826,SAT_AMEC_MX-GDL01,"[/proj/llc0156_zmx28home_3/q, /proj/llc0156_zm..."
2,RU61FPD561,udi.landen@LLp.com,"cn=LLa09278,ou=Personal,ou=People,ou=LLDI,o=LLP","nisMapName=auto.home,ou=RDC_AMEC_LL-CDC01,ou=L...",zru12,LLa09278,RDC_AMEC_LL-CDC01,"[/proj/llc2002_zru12/q, /proj/llc1002_zru12_an..."
3,KG63010285,adam.smith@LLp.com,NaN,"nisMapName=auto.home,ou=RDC_EMEA_NL-CDC01,ou=L...",Container to store ZuriKG vault,NaN,RDC_EMEA_NL-CDC01,[/proj/llc4008_zuriKG/q]
4,KG63010285,adam.smith@LLp.com,NaN,"nisMapName=auto.home,ou=RDC_EMEA_NL-CDC01,ou=L...",Project to restore project data on the RME wo...,NaN,RDC_EMEA_NL-CDC01,"[/proj/llc3008_zuriKG_rme/q, /proj/llc4014_zur..."


In [13]:
print(frame[["costCenter", "responsible", "directory"]])

   costCenter responsible                                          directory
0  LL63238012    LLf58420  [/proj/llc1002_ziz1/q, /proj/llc1002_ziz1_scra...
1  MX61FRK604    LLa11826  [/proj/llc0156_zmx28home_3/q, /proj/llc0156_zm...
2  RU61FPD561    LLa09278  [/proj/llc2002_zru12/q, /proj/llc1002_zru12_an...
3  KG63010285         NaN                           [/proj/llc4008_zuriKG/q]
4  KG63010285         NaN  [/proj/llc3008_zuriKG_rme/q, /proj/llc4014_zur...


In [8]:
print(frame.explode("directory")[["costCenter", "responsible", "directory"]])

   costCenter responsible                           directory
0  LL63238012    LLf58420                /proj/llc1002_ziz1/q
0  LL63238012    LLf58420        /proj/llc1002_ziz1_scratKG/q
1  MX61FRK604    LLa11826         /proj/llc0156_zmx28home_3/q
1  MX61FRK604    LLa11826           /proj/llc0156_zmx28home/q
1  MX61FRK604    LLa11826         /proj/llc0156_zmx28home_2/q
2  RU61FPD561    LLa09278               /proj/llc2002_zru12/q
2  RU61FPD561    LLa09278        /proj/llc1002_zru12_analog/q
2  RU61FPD561    LLa09278          /proj/llc1002_zru12_home/q
2  RU61FPD561    LLa09278        /proj/llc1002_zru12_libddk/q
2  RU61FPD561    LLa09278          /proj/llc1002_zru12_proj/q
3  KG63010285         NaN              /proj/llc4008_zuriKG/q
4  KG63010285         NaN          /proj/llc3008_zuriKG_rme/q
4  KG63010285         NaN          /proj/llc4014_zuriKG_rme/q
4  KG63010285         NaN  /proj/llc4014_zuriKG_rme_scratKG/q


In [9]:
#frame["username"] = frame["LLCAutoSudoRole"].str.extract("cn=([\w]*)")

In [10]:
frame

,costCenter,mail,LLpResponsible,LLpHomeDirectory,fullName,responsible,location,directory
0,LL63238012,shiva.gowni@LLp.com,"cn=LLf58420,ou=Personal,ou=People,ou=LLDI,o=LLP","nisMapName=auto.home,ou=RDC_AMEC_LL-CDC01,ou=L...",Tulip project ziz1,LLf58420,RDC_AMEC_LL-CDC01,"[/proj/llc1002_ziz1/q, /proj/llc1002_ziz1_scra..."
1,MX61FRK604,ali.pina@LLp.com,"cn=LLa11826,ou=Personal,ou=People,ou=LLDI,o=LLP","nisMapName=auto.home,ou=SAT_AMEC_MX-GDL01,ou=L...",xFSL to LLDI migration,LLa11826,SAT_AMEC_MX-GDL01,"[/proj/llc0156_zmx28home_3/q, /proj/llc0156_zm..."
2,RU61FPD561,udi.landen@LLp.com,"cn=LLa09278,ou=Personal,ou=People,ou=LLDI,o=LLP","nisMapName=auto.home,ou=RDC_AMEC_LL-CDC01,ou=L...",zru12,LLa09278,RDC_AMEC_LL-CDC01,"[/proj/llc2002_zru12/q, /proj/llc1002_zru12_an..."
3,KG63010285,adam.smith@LLp.com,NaN,"nisMapName=auto.home,ou=RDC_EMEA_NL-CDC01,ou=L...",Container to store ZuriKG vault,NaN,RDC_EMEA_NL-CDC01,[/proj/llc4008_zuriKG/q]
4,KG63010285,adam.smith@LLp.com,NaN,"nisMapName=auto.home,ou=RDC_EMEA_NL-CDC01,ou=L...",Project to restore project data on the RME wo...,NaN,RDC_EMEA_NL-CDC01,"[/proj/llc3008_zuriKG_rme/q, /proj/llc4014_zur..."
